In [39]:
import pandas as pd
import json
import datetime

In [6]:
file_path = "data/weather_data.json"

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [57]:
data_weather = pd.DataFrame()
start_date = datetime.datetime(2018, 8, 1)


for el in data:
    data_temp = el['main']
    timestamp = start_date
    start_date += timedelta(days=1)
    temp = data_temp['temp']-273
    press = data_temp['pressure']
    humid = data_temp['humidity']
    new_row = {'timestamp' : timestamp,'temp':temp,'press':press,'humid':humid}

    data_weather = data_weather.append(new_row, ignore_index=True)

C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1971967470.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1971967470.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1971967470.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1971967470.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dat

In [58]:
data_weather

,timestamp,temp,press,humid
0,2018-08-01,17.70,1008,50
1,2018-08-02,17.70,1008,50
2,2018-08-03,17.72,1008,50
3,2018-08-04,17.68,1008,50
4,2018-08-05,17.68,1008,50
...,...,...,...,...
411,2019-09-16,17.68,1008,50
412,2019-09-17,17.72,1008,50
413,2019-09-18,17.68,1008,50
414,2019-09-19,17.70,1008,50
